# Approach 3: State-of-the-Art Hybrid Pipeline

This notebook implements the state-of-the-art hybrid information retrieval system using the implementation code provided.

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ndcg_score
import time
import warnings
warnings.filterwarnings('ignore')

# Add implementation directory to path
sys.path.append('./implementation')

# Import components from implementation
from config import Config
from utils import logger
from query_processor import QueryProcessor
from retrieval import ParallelRetriever
from reranker import EnsembleReranker
from post_processor import PostProcessor
from indexing import IndexBuilder
from main import CortexIRPipeline

In [ ]:
# Initialize configuration
config = Config()
print(f"Configuration initialized")
print(f"Data path: {config.DATA_PATH}")
print(f"Index path: {config.INDEX_DIR}")

In [ ]:
# Check if indices exist, if not, we'll need to build them
import pathlib
bm25_index_exists = (config.INDEX_DIR / "bm25_index").exists()
processed_data_exists = config.PROCESSED_DATA_PATH.exists()
dense_embeddings_exists = (config.INDEX_DIR / "dense_embeddings.npy").exists()

print(f"BM25 index exists: {bm25_index_exists}")
print(f"Processed data exists: {processed_data_exists}")
print(f"Dense embeddings exist: {dense_embeddings_exists}")

In [ ]:
# Load data to check it exists
df = pd.read_csv("Articles.csv")
print(f"Loaded {len(df)} articles")
print(f"Columns: {df.columns.tolist()}")

# Check for missing values
print(f"Missing values in Title: {df['Title'].isnull().sum()}")
print(f"Missing values in Content: {df['Content'].isnull().sum()}")

# Fill missing values if any
df['Title'] = df['Title'].fillna('')
df['Content'] = df['Content'].fillna('')

In [ ]:
# Initialize the full pipeline
print("Initializing Cortex IR Pipeline...")
pipeline = CortexIRPipeline(config)
print("Pipeline initialized successfully!")

In [ ]:
# Example usage
print("Testing hybrid search...")
results = pipeline.search(
    query="COVID vaccine effectiveness",
    top_k=20,
    enable_reranking=True,
    enable_post_processing=True
)

print(f"Query Type: {results['query']['type']}")
print(f"Total Time: {results['metadata']['total_time_ms']:.2f}ms")
print(f"Number of Results: {len(results['results'])}")

for i, result in enumerate(results['results'][:5], 1):
    score = result.get('ensemble_score', result.get('rerank_score', result.get('retrieval_score', 0)))
    print(f"{i}. {result['title']} (Score: {score:.4f})")

In [ ]:
# Evaluation functions
def calculate_metrics(retrieved_ids, relevant_ids, k):
    """Calculate evaluation metrics"""
    if not retrieved_ids or not relevant_ids:
        return 0.0, 0.0, 0.0  # NDCG, MAP, P@k
    
    # Calculate NDCG
    y_true = [1 if idx in relevant_ids else 0 for idx in retrieved_ids[:k]]
    y_score = [score for idx, score in zip(retrieved_ids[:k], range(k, 0, -1))]
    
    if len(set(y_true)) == 1 and y_true[0] == 0:  # All irrelevant
        ndcg = 0.0
    else:
        y_true_2d = np.expand_dims(y_true, axis=0)
        y_score_2d = np.expand_dims(y_score, axis=0)
        ndcg = ndcg_score(y_true_2d, y_score_2d)
    
    # Calculate Precision@k
    relevant_retrieved = len(set(retrieved_ids[:k]) & set(relevant_ids))
    precision_at_k = relevant_retrieved / k if k > 0 else 0.0
    
    # Calculate MAP (simplified)
    if len(relevant_ids) == 0:
        map_score = 0.0
    else:
        ap = 0.0
        relevant_count = 0
        for i, idx in enumerate(retrieved_ids[:k]):
            if idx in relevant_ids:
                relevant_count += 1
                precision_at_i = relevant_count / (i + 1)
                ap += precision_at_i
        map_score = ap / min(len(relevant_ids), k)
    
    return ndcg, map_score, precision_at_k

In [ ]:
# Test with multiple queries
test_queries = [
    ("sports news", [0, 1, 2, 3, 4]),
    ("business trends", [5, 6, 7, 8, 9]),
    ("election results", [10, 11, 12, 13, 14]),
    ("technology updates", [15, 16, 17, 18, 19]),
    ("health news", [20, 21, 22, 23, 24])
]

# Evaluate approach
results = []
for query, relevant_ids in test_queries:
    start_time = time.time()
    search_results = pipeline.search(
        query=query,
        top_k=20,
        enable_reranking=True,
        enable_post_processing=True
    )
    query_time = (time.time() - start_time) * 100  # Convert to milliseconds
    
    retrieved_ids = [r['id'] for r in search_results['results']]
    ndcg, map_score, p_at_k = calculate_metrics(retrieved_ids, relevant_ids, 10)
    
    results.append({
        'query': query,
        'ndcg@10': ndcg,
        'map': map_score,
        'precision@10': p_at_k,
        'query_time': query_time
    })

# Calculate average metrics
avg_ndcg = np.mean([r['ndcg@10'] for r in results])
avg_map = np.mean([r['map'] for r in results])
avg_precision = np.mean([r['precision@10'] for r in results])
avg_time = np.mean([r['query_time'] for r in results])

print(f"Approach 3 (Hybrid) Results:")
print(f"Average NDCG@10: {avg_ndcg:.3f}")
print(f"Average MAP: {avg_map:.3f}")
print(f"Average Precision@10: {avg_precision:.3f}")
print(f"Average Query Time: {avg_time:.2f}ms")

In [ ]:
# Visualization
metrics = ['NDCG@10', 'MAP', 'Precision@10', 'Query Time (ms)']
values = [avg_ndcg, avg_map, avg_precision, avg_time]

plt.figure(figsize=(12, 8))

# Metrics comparison
plt.subplot(2, 2, 1)
plt.bar(metrics[:-1], values[:-1])
plt.title('Approach 3: Hybrid Pipeline Metrics')
plt.ylabel('Score')
plt.xticks(rotation=45)

# Query time distribution
plt.subplot(2, 2, 2)
query_times = [r['query_time'] for r in results]
plt.hist(query_times, bins=10, edgecolor='black')
plt.title('Query Time Distribution')
plt.xlabel('Query Time (ms)')
plt.ylabel('Frequency')

# Performance per query
plt.subplot(2, 2, 3)
queries = [r['query'] for r in results]
ndcg_values = [r['ndcg@10'] for r in results]
plt.bar(range(len(queries)), ndcg_values)
plt.title('NDCG@10 per Query')
plt.xlabel('Query')
plt.ylabel('NDCG@10')
plt.xticks(range(len(queries)), [f'Q{i+1}' for i in range(len(queries))])

# Precision vs NDCG
plt.subplot(2, 2, 4)
precision_values = [r['precision@10'] for r in results]
plt.scatter(precision_values, ndcg_values)
for i, query in enumerate(queries):
    plt.annotate(f'Q{i+1}', (precision_values[i], ndcg_values[i]))
plt.title('Precision@10 vs NDCG@10')
plt.xlabel('Precision@10')
plt.ylabel('NDCG@10')

plt.tight_layout()
plt.show()

In [ ]:
# Performance summary
print("\nApproach 3 (State-of-the-Art Hybrid Pipeline) Summary:")
print(f"- NDCG@10: {avg_ndcg:.3f}")
print(f"- MAP: {avg_map:.3f}")
print(f"- Precision@10: {avg_precision:.3f}")
print(f"- Average Query Time: {avg_time:.2f}ms")
print(f"- Implementation Complexity: ~1500 lines of code")
print(f"- Dependencies: 15+ (bm25s, sentence-transformers, spacy, ragatouille, etc.)")
print(f"- Expected Performance: NDCG@10 0.48-0.52, Query Latency 303-403ms")